<a href="https://colab.research.google.com/github/Luc1dddM/Wheather_WebSearch_ADK_Agent_LocalModel/blob/main/Wheather_Web_Search_Agetnt_Team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup and Installation
# Install ADK and LiteLLM for multi-model support

!pip install google-adk -q
!pip install litellm -q
!pip install playwright
!playwright install

print("Installation complete.")

167.7 MiB [] 0% 0.0s167.7 MiB [] 0% 32.8s167.7 MiB [] 0% 14.5s167.7 MiB [] 0% 6.5s167.7 MiB [] 1% 6.1s167.7 MiB [] 1% 5.8s167.7 MiB [] 2% 5.2s167.7 MiB [] 2% 4.9s167.7 MiB [] 3% 4.2s167.7 MiB [] 4% 3.9s167.7 MiB [] 4% 4.1s167.7 MiB [] 5% 4.1s167.7 MiB [] 6% 4.0s167.7 MiB [] 6% 3.7s167.7 MiB [] 7% 3.8s167.7 MiB [] 8% 3.6s167.7 MiB [] 8% 3.7s167.7 MiB [] 9% 3.5s167.7 MiB [] 10% 3.5s167.7 MiB [] 11% 3.3s167.7 MiB [] 11% 3.4s167.7 MiB [] 12% 3.2s167.7 MiB [] 13% 3.3s167.7 MiB [] 14% 3.2s167.7 MiB [] 15% 3.2s167.7 MiB [] 16% 3.2s167.7 MiB [] 17% 3.3s167.7 MiB [] 19% 3.3s167.7 MiB [] 20% 3.4s167.7 MiB [] 21% 3.4s167.7 MiB [] 22% 3.4s167.7 MiB [] 23% 3.3s167.7 MiB [] 24% 3.3s167.7 MiB [] 25% 3.3s167.7 MiB [] 25% 3.2s167.7 MiB [] 26% 3.3s167.7 MiB [] 27% 3.2s167.7 MiB [] 28% 3.2s167.7 MiB [] 29% 3.1s167.7 MiB [] 30% 3.0s167.7 MiB [] 31% 3.0s167.7 MiB [] 32% 3.0s167.7 MiB [] 33% 3.0s167.7 MiB [] 34% 3.1s167.7 MiB [] 35% 3.0s167.7 MiB [] 36% 3.0s167.7 MiB [] 38% 2.9s167.7 MiB [] 39% 2.9s167.7 Mi

In [ ]:
!pip install litellm

In [ ]:
# @title install ollama
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  48415      0 --:--:-- --:--:-- --:--:-- 48470
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
#Download model

!ollama pull llama3-groq-tool-use:8b

In [ ]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [ ]:
# --- Define Model Constants for easier use ---
MODEL_GWEN2_7B = "qwen2:7b"

In [ ]:
# @title Define the get_weather Tool
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

# Example tool usage (optional test)
print(get_weather("New York"))
print(get_weather("Paris"))

--- Tool: get_weather called for city: New York ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- Tool: get_weather called for city: Paris ---
{'status': 'error', 'error_message': "Sorry, I don't have weather information for 'Paris'."}


In [ ]:
from playwright.async_api import async_playwright
import asyncio

async def search_web_playwright(query: str):
    async with async_playwright() as p:
      #lauch browser
      brower = await p.chromium.launch(headless= True)

      #create a new tab
      page = await brower.new_page()

      await page.goto("https://www.bing.com/")

      await page.fill('textarea[name="q"]', query)
      await page.press('textarea[name="q"]', "Enter")

      # Đợi kết quả xuất hiện
      await page.wait_for_selector("#b_results", timeout=10000)

      # Lấy 3 kết quả đầu tiên
      results = []
      items = page.locator("li.b_algo h2 a").nth(0).locator("xpath=ancestor::li").locator("h2 a")
      links = page.locator("li.b_algo h2 a")

      count = await links.count()
      for i in range(min(3, count)):
          link = links.nth(i)
          title = await link.inner_text()
          href = await link.get_attribute("href")
          results.append({"title": title, "link": href})

      await page.close()
      return results

In [ ]:
# @title Define the search_web Tool
def search_web(search_term: str) -> list:
    """Retrieves the search results of search term.

    Args:
        search_term (str): user input.

    Returns:
        list: A list contant first three results of search.
              Includes a 'Title' key and 'Link' value.

    """
    try:
          # Run async function in event loop
          if asyncio.get_event_loop().is_running():
              # If already in async context, create new event loop
              import nest_asyncio
              nest_asyncio.apply()
              loop = asyncio.new_event_loop()
              asyncio.set_event_loop(loop)
              results = loop.run_until_complete(search_web_playwright(search_term))
              loop.close()
          else:
              # If not in async context, use normal run
              results = asyncio.run(search_web_playwright(search_term))

          # Format results as string for agent
          if results and len(results) > 0:
             return results
          else:
              return f"No results found for '{search_term}'"

    except Exception as e:
          return f"Search error: {str(e)}"

In [ ]:
# --- Web Search Agent ---
web_search_agent = None
try:
    web_search_agent = Agent(
        model = LiteLlm(model=f"ollama/{MODEL_GWEN2_7B}"),
        name="web_search_agent",
        instruction="You are the Web Search Agent. Your ONLY task is search something and provide infomation for user. "
                    "For all general questions (not about weather), use FunctionCall with name 'search_web' and args {'search_term': ...}"
                    "Use the 'search_web' tool to search about something. "
                    "If the user provides key work, make sure to pass it to the tool. "
                    "If the tool is successful, present the search result clearly."
                    "Do not engage in any other conversation or tasks.",
        description="Handles question about something of user",
        tools=[search_web],
    )
    print(f"✅ Agent '{web_search_agent.name}' created using model '{web_search_agent.model}'.")
except Exception as e:
    print(f"❌ Could not create Web search agent. Check API Key ({web_search_agent.model}). Error: {e}")



weather_agent = Agent(
    name="weather_agent_v1",
    model=LiteLlm(model=f"ollama/{MODEL_GWEN2_7B}"), # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

print(f"Agent '{weather_agent.name}' created using model '{MODEL_GWEN2_7B}'.")

# @title Define Greeting and Farewell Sub-Agents

# If you want to use models other than Gemini, Ensure LiteLlm is imported and API keys are set (from Step 0/2)
# from google.adk.models.lite_llm import LiteLlm
# MODEL_GPT_4O, MODEL_CLAUDE_SONNET etc. should be defined
# Or else, continue to use: model = MODEL_GEMINI_2_0_FLASH



✅ Agent 'web_search_agent' created using model 'model='ollama/qwen2:7b' llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x797aabfe4150>'.
Agent 'weather_agent_v1' created using model 'qwen2:7b'.


In [ ]:
# @title Define the Root Agent with Sub-Agents

# Ensure sub-agents were created successfully before defining the root agent.
# Also ensure the original 'get_weather' tool is defined.
root_agent = None
runner_root = None # Initialize runner
if web_search_agent and 'get_weather' in globals():
    weather_agent_team = Agent(
        name="weather_agent_v2", # Give it a new version name
        model=LiteLlm(model=f"ollama/{MODEL_GWEN2_7B}"),
        description="The main coordinator agent. Handles weather requests and delegates web search of specialists to sub agents.",
         instruction="""
              You are a coordinator agent. Your job is to route each user request to the correct sub-agent or tool.
              "You have specialized sub-agents: "
                    "1. 'web_search_agent': Handle user question when user ask you to search something "
              "Analyze the user's query. If user ask you to search something, delegate to 'web_search_agent'. "
              "If it's a request you search about something delegate to 'web_search_agent'. If it's a weather request, handle it yourself using 'get_weather'. "
              "For anything else, respond appropriately or state you cannot handle it."
              """,
        tools=[get_weather], # Root agent needs the weather tool for its core task
        # Key change: Link the sub-agents here!
        sub_agents=[web_search_agent], # Pass the agent instance, not the function,
        # before_model_callback=merge_text, # Optional callback function
    )
    print(f"✅ Root Agent '{weather_agent_team.name}' created using model '{MODEL_GWEN2_7B}' with sub-agents: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ Cannot create root agent because one or more sub-agents failed to initialize or 'get_weather' tool is missing.")
    if not web_search_agent: print(" - Web Search Agent is missing.")
    if 'get_weather' not in globals(): print(" - get_weather function is missing.")

ValidationError: 1 validation error for LlmAgent
  Value error, Agent `web_search_agent` already has a parent agent, current parent: `weather_agent_v2`, trying to add: `weather_agent_v2` [type=value_error, input_value={'name': 'weather_agent_v...er_tool_callback=None)]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
# @title Setup Session Service and Runner

# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=weather_agent_team, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)

print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='weather_tutorial_app', User='user_1', Session='session_001'
Runner created for agent 'weather_agent_v2'.


In [ ]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [ ]:
# @title Run the Initial Conversation

# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("What is the weather like in London?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("Search for me 'What is Google'",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # Expecting the tool's error message


# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()


>>> User Query: What is the weather like in London?
  [Event] Author: weather_agent_v2, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='call_d313660e-8c16-4d48-b6ab-c4c8d409bd7f', args={'city': 'London'}, name='get_weather'), function_response=None, text=None)] role='model'
--- Tool: get_weather called for city: London ---
  [Event] Author: weather_agent_v2, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='call_d313660e-8c16-4d48-b6ab-c4c8d409bd7f', name='get_weather', response={'status': 'success', 'report': "It's cloudy in London with a temperature of 15°C."}), text=None)] role=